In [1]:
import pandas as pd
import re
import difflib

1.	产品重复

In [2]:

# 读取CSV文件
df = pd.read_csv(r'output2\all_companies_merged_data.csv')
df_copy = df.copy()

# 删除'Product Model'列中包含'、'的行
df = df[~df['Product Model'].str.contains('、')]

def is_chinese(s):
    return any('\u4e00' <= char <= '\u9fff' for char in s)


# 过滤出包含非中文字符的行
df = df[df['company_name'].apply(is_chinese)]



In [3]:
skip_keywords_categories = (
    r'('
    # General terms related to root or system-on-chip
    r'root|soc|'
    # Download-related keywords
    r'下载|'
    # Development board-related terms
    r'开发板|開發板|'
    # Company names and general tech terms
    r'凡科网|凡科網|肖邦|code|'
    # Home page and index-related terms
    r'首页|首頁|位置|'
    # Certifications and awards
    r'证书|證書|大会|大會|博览会|博覽會|展会|展會|功能描述|产品型号|產品型號|查询|查詢|官网|官網|加特兰微电子|加特蘭微電子|腾振科技|騰振科技|'
    # Company descriptions and FAQs
    r'公司简介|公司簡介|常见问题|常見問題|爱心元速|愛心元速|凡科网，让经营更简单|凡科網，讓經營更簡單|'
    # Specific award mention
    r'"PlusTechnologyforwinningthe""202116thChinaCoreFireEmergingProductAward""!!"|'
    # News and updates
    r'资讯|資訊|动态|動態|企业|企業|网页|網頁|网站|網站|荣誉|榮譽|恭喜|庆祝|慶祝|资质|資質|关于我们|關於我們|案例|精品展示|展示|'
    # Social media and contact terms
    r'qq|微博|'
    # Product center and new releases
    r'产品中心|產品中心|新品发布|新品發布|'
    # Location and navigation terms
    r'Location|loctions|所有查询|所有查詢|解决方案|解決方案|芯路通讯科技有限公司|芯路通訊科技有限公司|二维码|二維碼|'
    # Marketing and promotional phrases
    r'new|congratulations|celebration|news|home|index|about|contact|location|新闻|新聞|祝贺|祝賀|'
    # Detailed product information
    r'详细信息|詳細信息|专注品质|專注品質|匠心工艺|匠心工藝|更自由|登录|登錄|查询查询|查詢查詢|华大电子|華大電子|China产品Factory,產品Supplier|获得|獲得|'
    # Product descriptions and lists
    r'产品中心|產品中心|产品展示|產品展示|产品介绍|產品介紹|产品列表|產品列表|产品目录|產品目錄|产品详情|產品詳情|产品系列|產品系列|产品型号|產品型號|产品规格|產品規格|'
    r'产品特点|產品特點|产品优势|產品優勢|产品功能|產品功能|产品参数|產品參數|产品图片|產品圖片|产品信息|產品信息|产品说明|產品說明|产品分类|產品分類|产品报价|產品報價|'
    r'产品价格|產品價格|产品销售|產品銷售|产品定制|產品定製|产品定位|產品定位|产品定价质量|產品定價質量|Kewell|Rising华兴|Rising華興|新品发布|新品發布|业务说明|業務說明|震撼发布|震撼發布|'
    # Homepage and promotion terms
    r'首頁|讓控制更簡單|ChinaProductsFactory,ProductsSupplier|language|'
    # File extensions
    r'.pdf|.doc|.docx|.xls|.xlsx|.ppt|.pptx|.txt|.zip|.rar|en\|us|(?i)slid\d+|'
    # Marketing and product descriptions
    r'获得专利分类导航|獲得專利分類導航|现货供应|現貨供應|概述|新品|先科|(?i)part\d+|新时代|新時代|美思迪赛|美思迪賽|全部产品|全部產品|又一个|又一個|'
    r'核心产品|核心產品|,,,,|芯导|芯導|恭贺|恭賀|引领|引領|深爱|深愛|Xrany元霓|'
    r'应用领域|應用領域|全系统解决方案|全系統解決方案|召开|召開|公司能力|行业解决方案|行業解決方案|KWansemi冠禹|'
    # Company details and partnerships
    r'资历|資歷|合作|伙伴|會談|莅临|蒞臨|'
    # Company abbreviations and titles
    r'Inc\.|Corp\.|Ltd\.|LLC|PLC|Co\.|GmbH|'
    # News and company product details
    r'报道|報導|介绍|介紹|公司产品|公司產品|Yourlocation|招聘|薪酬|人才|招贤纳士|招賢納士|喜讯|喜訊|荣获|榮獲|算法科学家|算法科學家|十年沉淀，六代匠心|十年沉澱，六代匠心|极昼系列的“前世今生”|極昼系列的“前世今生”|'
    # Generic phrases and sample code mentions
    r'更多|示例代码|示例代碼|了解|热爱|熱愛|激情|展示|精彩|便捷|下载中心|下載中心|列表|第(?:[0-9一二三四五六七八九十百千万]+)页|第(?:[0-9一二三四五六七八九十百萬]+)頁|'
    r'信昌|国科微|國科微|信浦|质量|質量|Kewell|更简单|更簡單|code|示例|zip.|使用说明|使用說明|国科微|國科微|信浦|七月上海|'
    # Welcome messages and company mentions
    r'Welcometowww.simgui.com.cn|TiandyTechnologies|取件码|取件碼|媒体中心|媒體中心|百强|百強|伟肯vacon，VACON代理，莱姆传感器|偉肯vacon，VACON代理，萊姆傳感器|'
    # Navigation and pagination terms
    r'下一页|下一頁|继往开来|繼往開來|爱科研|愛科研|Company|Exhibitioninvitation|website|'
    # No results found and common result count patterns
    r'未找到|相应参数|相應參數|共\d+条|共\d+條|共\d+页|共\d+頁|共\d+个|共\d+個|共\d+款|共\d+種|共\d+家|共\d+篇|共\d+条记录|共\d+條記錄|共\d+个结果|共\d+個結果|'
    r'共\d+个产品|共\d+個產品|共\d+个项目|共\d+個項目|共\d+个新闻|共\d+個新聞|共\d+个案例|共\d+個案例|共\d+个活动|共\d+個活動|共\d+个专题|共\d+個專題|'
    # Company-related terms and details
    r'芯片公司|芯片公司|Wuxii|Company|查看详情|查看詳情'
    r')'
)


2.	产品类别和型号需要拆分

In [4]:
# 定义过滤函数
def filter_categories_name(categories, company_name, skip_keywords_categories, model=None):
    filtered = []
    chinese_char_pattern = re.compile(r'[\u4e00-\u9fff]')
    url_pattern = re.compile(
        r'^(https?://)?([a-zA-Z0-9\-]+\.)+[a-zA-Z]{2,}(/[a-zA-Z0-9\-\._~:/?#@!$&\'()*+,;=]*)?$'
    )
    skip_keywords_pattern = re.compile(skip_keywords_categories, re.IGNORECASE)
    
    for c in categories:
        c = c.strip()
        if len(c) <= 1:  # 添加对字符长度的检查
            continue
        if skip_keywords_pattern.search(c):
            continue
        if c.lower().startswith("产品") or re.match(r'^(product|products)', c, re.IGNORECASE):
            continue
        if c.lower() == company_name.lower() or c.lower() in company_name.lower():
            continue
        
        similarity = difflib.SequenceMatcher(None, c.lower(), company_name.lower()).ratio()
        if similarity > 0.4:
            continue
        if model and model != '无':
            similarity1 = difflib.SequenceMatcher(None, c.lower(), model.lower()).ratio()
            if similarity1 > 0.8:
                continue
        
        chinese_chars = chinese_char_pattern.findall(c)
        if len(chinese_chars) > 10:
            continue

        if url_pattern.match(c):
            continue
        
        filtered.append(c)
    
    return filtered

# 过滤'root_category_name'列并处理分割后的列表
def process_root_category_name(row):

    if isinstance(row['root_category_name'], str):
        if '|' in row['root_category_name']:
            categories = row['root_category_name'].split('|')
# 对于每一个以 '|' 分割后的字符串，再进行 ',' 分割
            if ',' in row['root_category_name']:
                categories = [subcat for cat in categories for subcat in cat.split(',')]
        else:
            if ',' in row['root_category_name']:
                categories = row['root_category_name'].split(',')
            else:
             categories = [row['root_category_name']]
        
        # 去除非字符串类型的元素和纯数字字符串
        categories = [c for c in categories if isinstance(c, str) and not re.match(r'^\d+(\.\d+)?$', c.strip())]
        
        filtered_categories = filter_categories_name(categories, row['company_name'], skip_keywords_categories, row['Product Model'])
        if len(filtered_categories) == 2:
            row['root_category_name'] = filtered_categories[0]
            row['category_name'] = filtered_categories[1]
        elif len(filtered_categories) > 2:
            row['root_category_name'] = filtered_categories[-2]
            row['category_name'] = filtered_categories[-1]
        elif len(filtered_categories) == 1:
            row['root_category_name'] = filtered_categories[0]
            row['category_name'] = '无'
            row['catagory_id']=0
        else:
            row['root_category_name'] = '无'
            row['category_name'] = '无'
            row['catagory_id']=0
            row['root_catagory_id']=0
    else:
        row['root_category_name'] = '无'
        row['category_name'] = '无'
        row['catagory_id']=0
        row['root_catagory_id']=0
    return row

# 应用过滤和处理函数
df = df.apply(process_root_category_name, axis=1)


C:\Users\RAO SHENG\AppData\Local\Temp\ipykernel_12372\3896530907.py:8: DeprecationWarning: Flags not at the start of the expression '(root|soc|下载|开发板|開發板' (truncated) but at position 999
  skip_keywords_pattern = re.compile(skip_keywords_categories, re.IGNORECASE)
C:\Users\RAO SHENG\AppData\Local\Temp\ipykernel_12372\3896530907.py:8: DeprecationWarning: Flags not at the start of the expression '(root|soc|下载|开发板|開發板' (truncated) but at position 1048
  skip_keywords_pattern = re.compile(skip_keywords_categories, re.IGNORECASE)


移除包含en的url
如果移除英文页面后产品列表不为空


In [5]:


chinese_char_pattern = re.compile(r'[\u4e00-\u9fa5]')

# 检查字符串是否含有中文字符
def contains_chinese(text):
    if pd.isna(text):
        return False
    return bool(chinese_char_pattern.search(text))

# 找到符合删除条件的行的函数
def should_delete(row):
    product_model_condition = not contains_chinese(row['title'])
    base_info_condition = (not contains_chinese(row['base_info']))
    product_url_condition = (
        "/kr/" in row['product_url'] or 
        '/en.' in row['product_url'].lower() or 
        '/korea.' in row['product_url'].lower() or 
        '/en/' in row['product_url'].lower() or 
        '/korea/' in row['product_url'].lower()
    )

    # 检查三个条件中是否至少有两个为真
    conditions = [product_model_condition, base_info_condition, product_url_condition]
    return sum(conditions) >= 2 or ("/news." in row['product_url'] and "/news-" in row["product_url"])

# 初始化一个空的数据框
result_df = pd.DataFrame()

for name, group in df.groupby('company_name'):
    # 创建删除条件的掩码
    delete_mask = group.apply(should_delete, axis=1)
    
    # 临时删除符合条件的行
    temp_group = group[~delete_mask]
    
    if not temp_group.empty:
        result_df = pd.concat([result_df, temp_group], ignore_index=True)
    else:
        result_df = pd.concat([result_df, group], ignore_index=True)

df = result_df


In [6]:
df1 = pd.read_csv('eng_urls.csv')
url_list = df1['url'].tolist()


# 根据'product_url'列的值,删除匹配url_list中任意一个URL的整行
df = df[~df['product_url'].isin(url_list)]

# 将结果保存到新的CSV文件
df.to_csv('output.csv', index=False)

处理括号中的内容

In [7]:
def clean_name(name):
    # 定义匹配完整括号内容的正则表达式，包括《》和<>
    full_pattern = r'[\[【(《<].*?[\]】)》>]'
    left_pattern = r'[\[【(《<].*'  # 只有左括号及其之后的字符
    right_pattern = r'.*?[\]】)》>]'  # 只有右括号
    
    if re.search(full_pattern, name):
        # 如果匹配到完整的括号内容，删除它们
        return re.sub(full_pattern, '', name)
    elif re.search(left_pattern, name):
        # 如果只有左括号，删除左括号及其之后的字符
        return re.split(r'[\[【(《<]', name)[0]
    elif re.search(right_pattern, name):
        # 如果只有右括号，删除右括号
        return re.sub(r'[\]】)》>]', '', name)
    else:
        return name

    

df['root_category_name'] = df['root_category_name'].apply(clean_name)
df['category_name'] = df['category_name'].apply(clean_name)
df['sub_category_name'] = df['sub_category_name'].apply(clean_name)
df['detail_category_name'] = df['detail_category_name'].apply(clean_name)


C:\Users\RAO SHENG\AppData\Local\Temp\ipykernel_12372\3905335406.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['root_category_name'] = df['root_category_name'].apply(clean_name)
C:\Users\RAO SHENG\AppData\Local\Temp\ipykernel_12372\3905335406.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_name'] = df['category_name'].apply(clean_name)
C:\Users\RAO SHENG\AppData\Local\Temp\ipykernel_12372\3905335406.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

title clear

In [8]:

df = df[df['title'] != '无']
# 删除标题中包含关键词的行

id覆写

In [9]:
import hashlib
# 定义生成ID的函数
def generate_id(name):
    if name == '无':
        return 0
    else:
        return int(hashlib.md5(name.encode()).hexdigest(), 16) % (10 ** 10)

# 定义处理函数，移动下一级的名称到上一级
def handle_hierarchy(df):
    for i, row in df.iterrows():
        if row['root_category_name'] == '无':
            df.at[i, 'root_category_name'] = row['category_name']
            df.at[i, 'category_name'] = row['sub_category_name']
            df.at[i, 'sub_category_name'] = row['detail_category_name']
            df.at[i, 'detail_category_name'] = '无'
        elif row['category_name'] == '无':
            df.at[i, 'category_name'] = row['sub_category_name']
            df.at[i, 'sub_category_name'] = row['detail_category_name']
            df.at[i, 'detail_category_name'] = '无'
        elif row['sub_category_name'] == '无':
            df.at[i, 'sub_category_name'] = row['detail_category_name']
            df.at[i, 'detail_category_name'] = '无'

# 示例数据框

# 处理数据框
handle_hierarchy(df)

# 根据name生成ID
df['root_category_id'] = df['root_category_name'].apply(generate_id)
df['category_id'] = df['category_name'].apply(generate_id)
df['sub_category_id'] = df['sub_category_name'].apply(generate_id)
df['detail_category_id'] = df['detail_category_name'].apply(generate_id)

保存文档

In [10]:
columns_order = [
    'company_name', 'title', 'product_url', 'Product Model', 'base_info',
    'root_category_name', 'root_category_id', 'category_name', 'category_id',
    'sub_category_name', 'sub_category_id', 'detail_category_name', 'detail_category_id'
]
df =df[columns_order]


In [11]:
'''
# 去掉列名中的空格
df_copy.columns = df_copy.columns.str.strip()
df.columns = df.columns.str.strip()

# 定义要比较的列
compare_columns = ['title', 'Product Model', 'company_name']

# 使用布尔索引来找出 df_copy 中有但 df 中没有的行
# 通过创建一个唯一标识符列来比较
df_copy['unique_id'] = df_copy[compare_columns].astype(str).agg('-'.join, axis=1)
df['unique_id'] = df[compare_columns].astype(str).agg('-'.join, axis=1)

# 通过布尔索引找到 df_copy 中不在 df 中的行
not_in_df = df_copy[~df_copy['unique_id'].isin(df['unique_id'])]

# 删除临时列 'unique_id'
not_in_df = not_in_df.drop(columns=['unique_id'])

# 保存到CSV文件
not_in_df.to_csv(r'deleted.csv', index=False)

# 打印出这些行
print(not_in_df)


df =df[columns_order]
'''


"\n# 去掉列名中的空格\ndf_copy.columns = df_copy.columns.str.strip()\ndf.columns = df.columns.str.strip()\n\n# 定义要比较的列\ncompare_columns = ['title', 'Product Model', 'company_name']\n\n# 使用布尔索引来找出 df_copy 中有但 df 中没有的行\n# 通过创建一个唯一标识符列来比较\ndf_copy['unique_id'] = df_copy[compare_columns].astype(str).agg('-'.join, axis=1)\ndf['unique_id'] = df[compare_columns].astype(str).agg('-'.join, axis=1)\n\n# 通过布尔索引找到 df_copy 中不在 df 中的行\nnot_in_df = df_copy[~df_copy['unique_id'].isin(df['unique_id'])]\n\n# 删除临时列 'unique_id'\nnot_in_df = not_in_df.drop(columns=['unique_id'])\n\n# 保存到CSV文件\nnot_in_df.to_csv(r'deleted.csv', index=False)\n\n# 打印出这些行\nprint(not_in_df)\n\n\ndf =df[columns_order]\n"

base_info title 清理

In [12]:

keywords = ['手机QQ', '下载',"期待",'工程师','招聘',"未找到","报告","领军","研究院","解决方案","社会责任"]

for keyword in keywords:
    df = df[~df['title'].str.contains(keyword)]

# 定义删除HTML标签的函数
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 应用函数删除title中的HTML标签
df['title'] = df['title'].apply(remove_html_tags)

df['base_info'] = df['base_info'].apply(remove_html_tags)
for keyword in keywords:
    df = df[~df['base_info'].str.contains(keyword)]



In [13]:
df =df[columns_order]
df.to_csv(r'output2\all_companies_merged_data_new.csv', index=False)